In [ ]:
from __future__ import print_function 
import os, sys, h5py
import numpy as np
from six.moves import cPickle
from collections import OrderedDict

import tensorflow as tf
sys.path.append('..')
from tfomics import neuralnetwork as nn
from tfomics import neuralbuild as nb
from tfomics import utils, learn, explore

In [ ]:
def load_data(data_path):
    dataset = h5py.File(data_path,'r')

    print("loading training data")
    X_train = np.expand_dims(dataset['X_train'], axis=3).transpose([0,2,3,1])
    y_train = np.array(dataset['Y_train'])
    index = np.where(y_train[:,0]==0)[0]
    y = np.zeros((y_train.shape[0],1))
    y[index] = 1
    y_train = np.hstack([y_train, y])
    
    print("loading validation data")  
    X_valid = np.expand_dims(dataset['X_valid'], axis=3).transpose([0,2,3,1])
    y_valid = np.array(dataset['Y_valid'])
    index = np.where(y_valid[:,0]==0)[0]
    y = np.zeros((y_valid.shape[0],1))
    y[index] = 1
    y_valid = np.hstack([y_valid, y])

    print("loading test data")
    X_test = np.expand_dims(dataset['X_test'], axis=3).transpose([0,2,3,1])
    y_test = np.array(dataset['Y_test'])
    index = np.where(y_test[:,0]==0)[0]
    y = np.zeros((y_test.shape[0],1))
    y[index] = 1
    y_test = np.hstack([y_test, y])

    return X_train, y_train, X_valid, y_valid, X_test, y_test 


data_path = '/home/peter/harvard/genome/data'
file_name = 'K562_CEBPB_200_genome.h5'
X_train, y_train, X_valid, y_valid, X_test, y_test = load_data(os.path.join(data_path, file_name))

In [ ]:

def model(input_shape, num_labels):


    # create model
    layer1 = {  'layer': 'input',
                'input_shape': input_shape,
                }
    layer2 = {  'layer': 'conv1d', 
                'num_filters': {'start': 32, 'bounds': [1, 200], 'scale': 25, 'multiples': 4},
                'filter_size': {'start': 7, 'bounds': [5, 32], 'scale': 5, 'odd': True, 'multiples': 2},
                'norm': 'batch',
                'padding': 'SAME',
                'activation': 'relu',
                'dropout': {'start': 0.2, 'bounds': [0., 0.6], 'scale': 0.1},
                'name': 'conv1'
                }
    layer3 = {  'layer': 'conv1d', 
                'num_filters': {'start': 64, 'bounds': [1, 200], 'scale': 25, 'multiples': 4},
                'filter_size': {'start': 7, 'bounds': [5, 32], 'scale': 5, 'odd': True, 'multiples': 2},
                'norm': 'batch',
                'padding': 'SAME',
                'activation': 'relu',
                'dropout': {'start': 0.2, 'bounds': [0., 0.6], 'scale': 0.1},
                'name': 'conv2'
                }
    layer4 = {  'layer': 'dense', 
                'num_units': {'start': 256, 'bounds': [16, 1000], 'scale': 50, 'multiples': 4},
                #'norm': 'batch',
                'activation': 'relu',
                'dropout': {'start': 0.4, 'bounds': [0., 0.8], 'scale': 0.2},
                'name': 'dense1'
                }            
    layer5 = {  'layer': 'dense', 
                'num_units': num_labels,
                'activation': 'softmax',
                }

    #from tfomics import build_network
    model_layers = [layer1, layer2, layer3, layer4, layer5]

    # optimization parameters
    optimization = {"objective": "categorical",
                    "optimizer": "adam",
                    "learning_rate": 0.001,
                    "l2": 1e-6
                    #"learning_rate": {'start': -3, 'bounds': [-4, -1], 'scale': 1.5, 'transform': 'log'},      
                    #"l2": {'start': -6, 'bounds': [-8, -2], 'scale': 3, 'transform': 'log'},
                    # "l1": 0, 
                    }
    return model_layers, optimization

# build network
input_shape = list(X_train.shape)
input_shape[0] = None
num_labels = y_train.shape[1]
model_layers, optimization = model(input_shape, num_labels)

# build neural opti
optimizer = explore.NeuralOptimizer(model_layers, optimization)


In [ ]:
optimizer.sample_network()

In [ ]:
# optimize model parameters
train = {'inputs': [X_train], 'targets': y_train}
valid = {'inputs': [X_valid], 'targets': y_valid}        
optimizer.optimize(train, valid, num_trials=100, num_epochs=20, batch_size=128, verbose=0)

In [ ]:
# save optimal model
optimizer.print_optimal_model()